In [68]:
import openai
import requests
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity


In [69]:
API_KEY = "XXX"
RESOURCE_ENDPOINT = "XXX"

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"
r = requests.get(url, headers={"api-key": API_KEY})
print(r.text)  # see the models deployed in your account 



{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo-ozguler",
      "status": "succeeded",
      "created_at": 1679424805,
      "updated_at": 1679424805,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "text-davinci-003-ozguler",
      "status": "succeeded",
      "created_at": 1679425352,
      "updated_at": 1679425352,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1680255606,
      "updated_at": 1680255606,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scal

In [17]:
### text_columns = ['About Product', 'Product Specification', 'Technical Details', 'Ingredients', 'Direction To Use', 'Product Description']

In [70]:
import pandas as pd
df = pd.read_csv("./netflix_titles.csv")
df = df.loc[:, ["title", "description"]] 
df.shape
df.head()


,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [71]:
import re
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

df["description"] = list(map(lambda x: normalize_text(x), df["description"].values))
df.head()
#print(df["description"].values[1])

,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [73]:
#df["completions"][1:5] = df["description"].apply(lambda x : openai.ChatCompletion.create(messages = x, engine = 'gpt-35-turbo'))
df["embeddings"] = df["description"].apply(lambda x : get_embedding(x, engine = 'text-search-curie-doc-001'))

In [74]:
df["embeddings"]


0       [0.0043516214936971664, 0.0022802685853093863,...
1       [-0.02388826385140419, 0.0029224504251033068, ...
2       [-0.034246984869241714, 0.010515750385820866, ...
3       [-0.02306009829044342, -0.008633919060230255, ...
4       [-0.030220653861761093, 0.009630915708839893, ...
                              ...                        
8802    [-0.022079553455114365, 0.00714001152664423, -...
8803    [-0.025093426927924156, -0.007184464950114489,...
8804    [-0.02202506549656391, -0.007789113558828831, ...
8805    [-0.01231981161981821, -0.01197921670973301, 0...
8806    [-0.016202343627810478, 0.0025977350305765867,...
Name: embeddings, Length: 8807, dtype: object

In [84]:
# search through the descriptions for a specific movie title / descripton
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-search-curie-query-001"
    )

    df["similarities"] = df.embeddings.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df, "A young Han Solo tries to settle an old score with the help of his new buddy Chewbacca, a crew of space smugglers and a cunning old friend.", top_n=10)

,title,description,embeddings,similarities
8052,Solo: A Star Wars Story (Spanish Version),A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.544304
8051,Solo: A Star Wars Story,A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.544304
594,Star Trek,On their first voyage aboard the starship Ente...,"[-0.020636947825551033, -0.001046766759827733,...",0.362261
2950,Dragonheart: Vengeance,"When his family is slain by vicious savages, a...","[-0.009340716525912285, 0.014154045842587948, ...",0.353338
1239,Parker,Parker is a thief – but he has scruples. So wh...,"[-0.02055243030190468, -0.006717955693602562, ...",0.348927
8056,Son of a Gun,A young petty thief who received protection fr...,"[-0.022929834201931953, 9.639817290008068e-05,...",0.344896
4014,Robozuna,A boy and his homemade robot enter the toughes...,"[-0.00838391575962305, 0.010517490096390247, 0...",0.343526
4456,Raja Natwarlal,A small-time con man assembles a team to help ...,"[-0.010686958208680153, -0.0005034700152464211...",0.343398
6733,Fakta Ladh Mhana,A young man enlists the help of his friends to...,"[-0.004655837547034025, 0.0034520288463681936,...",0.342674
6654,Dum,When one of his colleagues becomes corrupted b...,"[-0.0041526020504534245, -0.010855847969651222...",0.342658


In [86]:
title = "Star Trek"
description = df.loc[df["title"] == title, "description"].iloc[0]
res = search_docs(df, description, top_n=15)

,title,description,embeddings,similarities
594,Star Trek,On their first voyage aboard the starship Ente...,"[-0.020636947825551033, -0.001046766759827733,...",0.568379
4946,Star Trek: The Next Generation,Decades after the adventures of the original E...,"[-0.01778496243059635, -0.009458758868277073, ...",0.380679
5244,Star Trek: Enterprise,Capt. Archer and his crew explore space and di...,"[-0.012841334566473961, -0.005552239716053009,...",0.376742
8051,Solo: A Star Wars Story,A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.354305
8052,Solo: A Star Wars Story (Spanish Version),A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.354305
3187,Nightflyers,"With humankind's future at stake, a group of s...","[-0.01391106378287077, -0.022113019600510597, ...",0.350297
4014,Robozuna,A boy and his homemade robot enter the toughes...,"[-0.00838391575962305, 0.010517490096390247, 0...",0.348702
955,Under Siege,A disgruntled former CIA operative and his gro...,"[-0.005343559198081493, 0.0074609084986150265,...",0.348158
1178,Jiu Jitsu,"After the defeat of a celebrated war hero, an ...","[-0.008127129636704922, -0.002662672195583582,...",0.346033
437,Cosmic Sin,"In the year 2524, a disgraced retired general ...","[0.003747612703591585, 4.616237129084766e-05, ...",0.344804


In [ ]:
title = "Star Trek"
description = df.loc[df["title"] == title, "description"].iloc[0]
res = search_docs(df, description, top_n=15)

,title,description,embeddings,similarities
594,Star Trek,On their first voyage aboard the starship Ente...,"[-0.020636947825551033, -0.001046766759827733,...",0.568379
4946,Star Trek: The Next Generation,Decades after the adventures of the original E...,"[-0.01778496243059635, -0.009458758868277073, ...",0.380679
5244,Star Trek: Enterprise,Capt. Archer and his crew explore space and di...,"[-0.012841334566473961, -0.005552239716053009,...",0.376742
8051,Solo: A Star Wars Story,A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.354305
8052,Solo: A Star Wars Story (Spanish Version),A young Han Solo tries to settle an old score ...,"[-0.01390331145375967, -0.0010706261964514852,...",0.354305
3187,Nightflyers,"With humankind's future at stake, a group of s...","[-0.01391106378287077, -0.022113019600510597, ...",0.350297
4014,Robozuna,A boy and his homemade robot enter the toughes...,"[-0.00838391575962305, 0.010517490096390247, 0...",0.348702
955,Under Siege,A disgruntled former CIA operative and his gro...,"[-0.005343559198081493, 0.0074609084986150265,...",0.348158
1178,Jiu Jitsu,"After the defeat of a celebrated war hero, an ...","[-0.008127129636704922, -0.002662672195583582,...",0.346033
437,Cosmic Sin,"In the year 2524, a disgraced retired general ...","[0.003747612703591585, 4.616237129084766e-05, ...",0.344804


In [63]:
from azure.cosmos import CosmosClient

In [65]:
endpoint = 'https://cosmos-openai-ozguler.documents.azure.com:443/'
key = 'zotExgWvfvXVWD9ROscNtr6A0wrJ3COYhYBGqMi8R3OPEBDJEvhXdtnnDbRsMRuQs1a0z6h3ntgJACDbWIvkVQ=='
client = CosmosClient(endpoint, key)

In [64]:
df

,title,description,embeddings,similarities
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...","[0.0043516214936971664, 0.0022802685853093863,...",0.290345
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[-0.02388826385140419, 0.0029224504251033068, ...",0.266189
2,Ganglands,To protect his family from a powerful drug lor...,"[-0.034246984869241714, 0.010515750385820866, ...",0.326060
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo...","[-0.02306009829044342, -0.008633919060230255, ...",0.248097
4,Kota Factory,In a city of coaching centers known to train I...,"[-0.030220653861761093, 0.009630915708839893, ...",0.261389
...,...,...,...,...
8802,Zodiac,"A political cartoonist, a crime reporter and a...","[-0.022079553455114365, 0.00714001152664423, -...",0.274146
8803,Zombie Dumb,"While living alone in a spooky town, a young g...","[-0.025093426927924156, -0.007184464950114489,...",0.294525
8804,Zombieland,Looking to survive in a world taken over by zo...,"[-0.02202506549656391, -0.007789113558828831, ...",0.304413
8805,Zoom,"Dragged from civilian life, a former superhero...","[-0.01231981161981821, -0.01197921670973301, 0...",0.301935
